In [7]:
import os
import random
import bisect
import warnings

import numpy        as np
import pandas       as pd
import scipy.stats  as ss

import seaborn           as sns
from matplotlib          import pyplot as plt
from matplotlib.gridspec import GridSpec

from IPython.display         import Image
from IPython.core.display    import HTML

from imblearn                import combine        as c

from pandas_profiling        import ProfileReport

from sklearn.metrics         import (accuracy_score, roc_auc_score, average_precision_score, precision_score,
                                    recall_score, f1_score, cohen_kappa_score, balanced_accuracy_score, classification_report)
from sklearn.ensemble        import RandomForestClassifier
from sklearn.neighbors       import KNeighborsClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.preprocessing   import OneHotEncoder
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split, StratifiedKFold

from xgboost                 import XGBClassifier
from lightgbm                import LGBMClassifier
from catboost                import CatBoostClassifier

from skopt                   import forest_minimize

from boruta                  import BorutaPy

from scikitplot.metrics      import plot_confusion_matrix


# import sys
# sys.path.append('../')
# ## My Functions
# from modules.postgres import Postgres


from sqlalchemy import create_engine, inspect

In [8]:
host = 'comunidade-ds-postgres.c50pcakiuwi3.us-east-1.rds.amazonaws.com'
usenamer = os.environ.get('POSTGRES_CDS_USERNAME')
password = os.environ.get('POSTGRES_CDS_PASSWORD')
port = '5432'
database = 'comunidadedsdb'

engine = create_engine(f"postgresql://{usenamer}:{password}@{host}:{port}/{database}")

inspector = inspect(engine)

# get schemas
schemas = inspector.get_schema_names()

for schema in schemas:
    if (schema != 'information_schema') & (schema != 'public'):
        # list of tables in each schema
        print('schema: {}'.format(schema))
        tables = inspector.get_table_names(schema=schema)
        for table in tables:
            print('table: {}'.format(table))

schema: pa004
table: users
table: vehicle
table: insurance


In [13]:
query_raw_data = sql

with engine.connect() as conn:
    df_raw = pd.read_sql(query_raw_data, conn)

In [14]:
df_raw.head()

,id,gender,age,region_code,policy_sales_channel,driving_license,vehicle_age,vehicle_damage,previously_insured,annual_premium,vintage,response
0,7,Male,23,11.0,152.0,1,< 1 Year,Yes,0,23367.0,249,0
1,13,Female,41,15.0,14.0,1,1-2 Year,No,1,31409.0,221,0
2,18,Female,25,35.0,152.0,1,< 1 Year,No,1,46622.0,299,0
3,31,Female,26,8.0,160.0,1,< 1 Year,No,0,2630.0,136,0
4,39,Male,45,8.0,124.0,1,1-2 Year,Yes,0,42297.0,264,0


In [11]:
with open('../02-Query/raw_data.sql', mode='r', encoding='utf-8-sig') as query:
    sql = query.read()

In [3]:
sql = open('../02-Query/raw_data.sql', mode='r', encoding='utf-8-sig').read()

In [12]:
sql

'SELECT u.id\n                        ,u.gender\n                        ,u.age\n                        ,u.region_code\n                        ,u.policy_sales_channel\n                        ,v.driving_license\n                        ,v.vehicle_age\n                        ,v.vehicle_damage\n                        ,i.previously_insured\n                        ,i.annual_premium\n                        ,i.vintage\n                        ,i.response\n               FROM pa004.users AS u \n               LEFT JOIN pa004.vehicle AS v ON u.id = v.id \n               LEFT JOIN pa004.insurance AS i ON u.id = i.id'